In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # 検索ツールの定義
    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    
    # グラフのインスタンスを作成
    graph_builder = StateGraph(State)

    # 言語モデルの定義
    llm = ChatOpenAI(model_name=model_name)

    # ツール定義の紐づけ
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノードの作成
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    # グラフにチャットボットノードを追加
    graph_builder.add_node("chatbot", chatbot)

    # ツールノードの作成
    tool_node = ToolNode(tools)

    # グラフにツールノードを追加
    graph_builder.add_node("tools", tool_node)

    # 条件付エッジの作成
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition, # ツール呼出と判断したらツールノードを呼ぶ
    )

    # ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
    # ツールからチャットボットへの戻りエッジを作成
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノードの指定
    graph_builder.set_entry_point("chatbot")

    return graph_builder

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    return events
    
# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
memory = MemorySaver()
graph = build_graph(MODEL_NAME).compile(checkpointer=memory)

# メインループ
# チャットボットのループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    events = stream_graph_updates(graph, user_input)
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？
1たす2は？
1たす2は3です。何か他に知りたいことがありますか？
台湾観光について検索結果を教えて

[{"url": "https://www.kkday.com/ja/blog/48762/taiwan-sightseeing?srsltid=AfmBOoqIh-NqpXwnYOHh46KsSyC7xiRAiM9120r0odFkZeX85vk9Gfqx", "content": "台湾観光には台北市政府が発行している「台北無限周遊パス」の利用がおすすめ！有名な観光スポットに無料で入場できたり、公共交通機関が乗り放題になったりと、お得な特典が盛り沢山です。\n\n1 / 2 / 3日券があるため、自分のスケジュールにあったプランを選択できるのも魅力。KKdayで事前予約を済ましておけば、空港やMRT台北駅で簡単に受け取りができます。\n\n台湾観光をお得に楽しむために、ぜひ台北無限周遊パスをチェックしてみてください。\n\n## 台湾・台北のおすすめ観光スポット16選\n\n### 1. 台北101｜台北の街並みを一望できる台湾の観光名所\n\n台湾　観光\n\n最初に紹介する台湾のおすすめ観光スポットは「台北101」。高さ約500mを誇る、台湾で一番高いビルです。101階建てであることから名付けられたといわれています。魅力ポイントは、展望台からは台北の美しい街並みを一望できること。\n\nさらにビル内にはレストランやカフェ、ブランドショップなどがたくさん入っているので、買い物も合わせて楽しめます。KKdayでは入場チケットを割引価格で販売しているので、これから行く方は必見です！ [...] 台湾 観光\n\n続いて紹介する台湾のおすすめ観光スポットは「彩虹眷村（レインボービレッジ）」。彩虹眷村は、辺り一体がカラフルなイラストで埋め尽くされた若者に人気のフォトジェニックスポットです。見ているだけで元気になれるような色使いで、SNS映え間違いなし！\n\n彩虹眷村の絵は全て香港九龍出身の黄永阜さんが一人で描いたもので、趣味としてコツコツと描いたそうです。併設されている売店にはポストカードやTシャツ、アイスキャンディーなどが売られているので、ぜひ立ち寄ってみてください。\n\n彩虹眷村や高美湿地など